In [122]:
from torch.nn import RNN ,GRU , LSTM 
import torch.nn as nn
import torch
from torch.utils.data import DataLoader , Dataset
import torch.optim as optim
import pandas  as pd

# Load Sequance data sets 

In [75]:
# Load the CSV data
train_data = pd.read_csv('electricity_train.csv')
test_data = pd.read_csv('electricity_test.csv')

# Display the first few rows to understand the data
print(train_data.head())
print(test_data.head())

             timestamp  consumption
0  2011-01-01 00:15:00    -0.704319
1  2011-01-01 00:30:00    -0.704319
2  2011-01-01 00:45:00    -0.678983
3  2011-01-01 01:00:00    -0.653647
4  2011-01-01 01:15:00    -0.704319
             timestamp  consumption
0  2014-01-01 00:00:00    -0.932595
1  2014-01-01 00:15:00    -0.957931
2  2014-01-01 00:30:00    -0.932595
3  2014-01-01 00:45:00    -0.907259
4  2014-01-01 01:00:00    -0.881923


In [46]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105215 entries, 0 to 105214
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   timestamp    105215 non-null  object 
 1   consumption  105215 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.6+ MB


In [151]:
import numpy as np
seq_length = 24
def create_sequences(df, seq_length):
    xs, ys = [], []
    for i in range(len(df) - seq_length):
        x = df.iloc[i:(i+seq_length), 1]
        y = df.iloc[i+seq_length, 1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

In [157]:
# Create sequences for the training data
X_train, y_train = create_sequences(train_data,seq_length)
print(X_train.shape, y_train.shape)

(105191, 24) (105191,)


In [158]:
X_test, y_test = create_sequences(test_data, seq_length)
print(X_test.shape, y_test.shape)

(35016, 24) (35016,)


In [159]:
# Convert the sequences to PyTorch tensors
from torch.utils.data import TensorDataset
dataset_train = TensorDataset(
                torch.from_numpy(X_train).float(),
                torch.from_numpy(y_train).float(),
                )

dataset_test = TensorDataset(
                torch.from_numpy(X_test).float(),
                torch.from_numpy(y_test).float(),
                )

In [160]:
# Create DataLoaders
dataloader_train = DataLoader(dataset_train, batch_size=32, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=32, shuffle=False)

# Anthor way for loading sequance data 

In [46]:
# Create a custom dataset class for PyTorch
class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [48]:
# Convert the sequences to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [50]:
# Create train and test datasets
train_dataset = SequenceDataset(X_train_tensor, y_train_tensor)
test_dataset = SequenceDataset(X_test_tensor, y_test_tensor)

In [52]:
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [54]:
# Example: Iterate through the DataLoader
for batch in train_loader:
    features, labels = batch
    print(features)
    print(labels)
    break  # To only print the first batch

tensor([[-1.1862, -1.1609, -1.1355, -1.0342, -0.9326, -0.9579, -0.9326, -0.9326,
         -0.9326, -0.9579],
        [ 0.9501,  0.4160,  0.3398,  0.1873,  0.2128,  0.2128,  0.1873, -0.7043,
         -0.6283, -0.6790],
        [ 1.4330,  1.3314,  1.0773,  1.2298,  1.4330,  1.5093,  1.4839,  1.0517,
          0.7720,  0.6957],
        [ 1.8797,  1.8288,  1.8544,  1.9053,  0.2891, -0.2734, -0.2734, -0.3240,
         -0.6030, -0.6283],
        [-0.8819, -0.8819, -0.8566, -0.8059,  0.0816,  1.0008,  0.9755,  0.7467,
          0.6451,  0.5941],
        [ 1.5602,  1.8797,  1.9053,  1.8544,  2.0074,  2.1097,  2.1097,  2.0841,
          2.0586,  1.9820],
        [ 0.1579,  0.1069,  0.0816,  0.0056,  0.0056, -0.0451,  0.0309,  0.0563,
          0.1325,  0.1325],
        [ 0.0563, -0.0197,  0.1873,  0.5688,  0.5432,  0.7467,  0.9245,  0.7976,
          0.8229,  0.7213],
        [-0.8819, -0.8819, -0.8059, -0.8819, -0.8819, -0.8313, -0.8566, -0.8566,
         -0.8059, -0.8059],
        [-1.0342, -

# RNN in PyTorch

In [319]:

class RNNModel(nn.Module):
    def __init__(self):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(
            input_size=1,    # Input feature size
            hidden_size=32,  # Hidden state size
            num_layers=2,    # Number of RNN layers
            batch_first=True  # Input shape: (batch, seq, feature)
        )
        self.fc = nn.Linear(32, 1)  # Output layer

    def forward(self, x):
        # Initialize hidden state
        h0 = torch.zeros(2, x.size(0), 32)  # (num_layers, batch_size, hidden_size)
        
        # Forward propagate RNN
        out, _ = self.rnn(x, h0)  # out shape: (batch_size, seq_length, hidden_size)

        # Pass the last time step's output through the linear layer
        out = self.fc(out[:, -1, :])  # Taking the output from the last time step
        return out


# LSTM in PyTorch

In [59]:
class lstm(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.lstm = nn.LSTM(
        input_size=1,
        hidden_size=32,
        num_layers=2,
        batch_first=True,
        )
        self.fc = nn.Linear(32, 1)
    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 32)
        c0 = torch.zeros(2, x.size(0), 32)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# GRU in PyTorch

In [62]:
class gru(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.gru = nn.GRU(
        input_size=1,
        hidden_size=32,
        num_layers=2,
        batch_first=True,
        )
        self.fc = nn.Linear(32, 1)
    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 32)
        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :])
        return out

# Training loop

In [167]:
for seqs, labels in dataloader_train :
    print(seqs.shape)

torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([32, 24])
torch.Size([3

In [190]:
seq.size(0)

32

In [192]:
seq.numel() // batch_size

24

In [198]:
net= RNNModel()
# preform loss and optimizer 
critioen = nn.MSELoss()
optimizer = optim.Adam(net.parameters()) 
#Training loop
for epoch in range(5):
    for seq , labels in dataloader_train:
        batch_size = seq.size(0)
        sequence_length = seq.numel() // batch_size
        seq = seq.view(batch_size, sequence_length, 1)  # Reshape for RNN input
        out = net(seq)
        if labels.dim() == 1:  # Check if labels need reshaping
            labels = labels.unsqueeze(1)  # Reshape to (batch_size, 1)
        loss = critioen(out,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.6036519408226013
Epoch 2, Loss: 0.023278797045350075
Epoch 3, Loss: 0.07575679570436478
Epoch 4, Loss: 0.004364077467471361
Epoch 5, Loss: 0.008573109284043312


# Evaluation loop

In [215]:
import torchmetrics

In [327]:
# Initialize the Mean Squared Error metric
mse = torchmetrics.MeanSquaredError()

# Set the model to evaluation mode
net.eval()
with torch.no_grad():  # Disable gradient calculation for evaluation
    for seqs, labels in dataloader_test:
        # Check the shape of the input tensor
        print(f'Shape of input sequences (seqs): {seqs.shape}')  # Should be (batch_size, seq_length, 1)

        # Ensure that seqs are in the right shape for RNN
        if seqs.dim() != 3:
            seqs = seqs.view(-1, seqs.size(1), 1)  # Reshape if needed

        # Pass the input sequences to the model
        outputs = net(seqs).squeeze()  # Output will be of shape (batch_size)

        # Check the shape of the outputs and labels
        print(f'Shape of outputs: {outputs.shape}')  # Should be (batch_size,)
        print(f'Shape of labels: {labels.shape}')    # Should be (batch_size,)

        # Ensure labels are in the right shape (1D)
        if labels.dim() > 1:
            labels = labels.squeeze()  # Make sure labels are 1D

        # Update the MSE metric
        mse(outputs, labels)

# Compute and print the final MSE
final_mse = mse.compute()
print(f"Test Mean Squared Error: {final_mse.item()}")


Shape of input sequences (seqs): torch.Size([32, 24])
Shape of outputs: torch.Size([32])
Shape of labels: torch.Size([32])
Shape of input sequences (seqs): torch.Size([32, 24])
Shape of outputs: torch.Size([32])
Shape of labels: torch.Size([32])
Shape of input sequences (seqs): torch.Size([32, 24])
Shape of outputs: torch.Size([32])
Shape of labels: torch.Size([32])
Shape of input sequences (seqs): torch.Size([32, 24])
Shape of outputs: torch.Size([32])
Shape of labels: torch.Size([32])
Shape of input sequences (seqs): torch.Size([32, 24])
Shape of outputs: torch.Size([32])
Shape of labels: torch.Size([32])
Shape of input sequences (seqs): torch.Size([32, 24])
Shape of outputs: torch.Size([32])
Shape of labels: torch.Size([32])
Shape of input sequences (seqs): torch.Size([32, 24])
Shape of outputs: torch.Size([32])
Shape of labels: torch.Size([32])
Shape of input sequences (seqs): torch.Size([32, 24])
Shape of outputs: torch.Size([32])
Shape of labels: torch.Size([32])
Shape of input s